In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [535]:
def calculate_metrics(model, loader):
    model.eval()
        
    with torch.no_grad():
        outputs = model(loader.val_data_tensor, loader.raw_output_val)
        
        _, preds = torch.max(outputs, 1)
        print(outputs.shape)
        
        all_preds = preds.cpu().numpy()
        all_labels = loader.val_labels_tensor.cpu().numpy()
    
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return accuracy, f1

In [474]:
class CustomDataLoader:
    def __init__(self, features, labels, batch_size=1, validation_size=0.0, raw_output=None):
        train_indices, val_indices, train_labels, val_labels = train_test_split(
            range(len(labels)), labels, test_size=0.2, stratify=labels, random_state=42
        )
    
        train_data = features[train_indices]
        val_data = features[val_indices]

        self.train_data_tensor = torch.tensor(train_data).float().to(device)
        self.train_labels_tensor = torch.tensor(train_labels).long().to(device)
        
        if raw_output is None:
            self.raw_output_train = None
            self.raw_output_val = None
        else:
            self.raw_output_train = raw_output[train_indices].clone().detach().float().to(device)
            self.raw_output_val = raw_output[val_indices].clone().detach().float().to(device)

        self.val_data_tensor = torch.tensor(val_data).float().to(device)
        self.val_labels_tensor = torch.tensor(val_labels).long().to(device)

        train_dataset = TensorDataset(self.train_data_tensor, self.train_labels_tensor)
        val_dataset = TensorDataset(self.val_data_tensor, self.val_labels_tensor)

        self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        self.val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    def get_train_loader(self):
        return self.train_loader
    
    def get_val_loader(self):
        return self.val_loader

In [475]:
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, batch_size, patience=None):
    best_val_loss, early_val_accuracy, early_train_loss = float('inf'), 0.0, float('inf')
    best_epoch = 0
    patience_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i in range(0, custom_train_loader.train_data_tensor.size(0), batch_size):
            inputs, labels = custom_train_loader.train_data_tensor[i:i + batch_size], custom_train_loader.train_labels_tensor[i:i + batch_size]
            optimizer.zero_grad()
            
            outputs = None
            if custom_train_loader.raw_output_train is None:
                outputs = model(inputs.view(-1, num_features))
            else:
                outputs = model(inputs.view(-1, num_features), custom_train_loader.raw_output_train[i:i + batch_size])
                
            loss = criterion(outputs, labels, model)
            running_loss += loss.item()
    
            loss.backward()
            optimizer.step()

        model.eval()
        running_val_loss = 0.0

        unregularized_criterion=nn.CrossEntropyLoss()
        with torch.no_grad():
            for i in range(0, custom_train_loader.val_data_tensor.size(0), batch_size):
                inputs, labels = custom_train_loader.val_data_tensor[i:i + batch_size], custom_train_loader.val_labels_tensor[i:i + batch_size]
                inputs, labels = inputs.to(device), labels.to(device)
                
                if custom_train_loader.raw_output_train is None:
                    outputs = model(inputs.view(-1, num_features))
                else:
                    outputs = model(inputs.view(-1, num_features), custom_train_loader.raw_output_val[i:i + batch_size])

                val_loss = unregularized_criterion(outputs, labels)
                running_val_loss += val_loss.item()

            avg_val_loss = running_val_loss / (len(custom_train_loader.get_val_loader()))

            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader)

            if (epoch + 1) % 1 == 0:
                print(f'Epoch {epoch+1}, Loss: {running_loss / (len(custom_train_loader.get_train_loader()))}')
                print(f'Validation Loss: {avg_val_loss}')
                print(f'Val Accuracy: {val_accuracy:.4f}, Val F1-score: {val_f1:.4f}')
                print()

            if patience is None:
                continue
            elif avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                early_val_accuracy = val_accuracy
                early_train_loss = running_loss / len(custom_train_loader.get_train_loader())
                best_epoch = epoch
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Early stopping triggered. Best validation loss at epoch {best_epoch+1}: {best_val_loss:.4f}')
                print(f'Val Accuracy: {early_val_accuracy:.4f}')
                print(f'Early Train Loss: {early_train_loss}')
                break

In [476]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, l1_lambda, l2_lambda):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    def forward(self, outputs, labels, model):
        loss = self.criterion(outputs, labels)
        
        l1_norm = sum(p.abs().sum() for name, p in model.named_parameters() if 'bias' not in name)
        l2_norm = sum(p.pow(2.0).sum() for name, p in model.named_parameters() if 'bias' not in name)
        
        loss += self.l1_lambda * l1_norm + self.l2_lambda * l2_norm
        return loss

In [477]:
data = pd.read_csv('/kaggle/input/adult-income-dataset/adult.csv')
data = data.dropna()

X = data.drop(["income"], axis=1)
y = data["income"]

X = pd.get_dummies(X, drop_first=True)
for col in X.columns:
    if (X[col] > 0).all():
        X[col] = np.log(X[col])

print(X.shape)

(48842, 100)


In [478]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(y_encoded)

[0 0 1 ... 0 0 1]


In [491]:
class CustomLinearLayer(nn.Module):
    def __init__(self, input_size, output_size, init="looks_linear"):
        super(CustomLinearLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size, bias=True)
        nn.init.zeros_(self.linear.bias)

        if init == "zero":
            nn.init.zeros_(self.linear.weight)
        elif init == "looks_linear":
            if input_size * 2 != output_size:
                raise ValueError("Output size must be twice that of input size")
                
            with torch.no_grad():
                weight = torch.zeros(input_size * 2, input_size)

                for i in range(self.linear.in_features):
                    weight[2 * i, i] = 1
                    weight[2 * i + 1, i] = -1

                self.linear.weight.copy_(weight)
                nn.init.zeros_(self.linear.bias)
                
            """ Example matrix: [
                [1, 0, 0],
                [-1, 0, 0],
                [0, 1, 0],
                [0, -1, 0],
                [0, 0, 1],
                [0, 0, -1]
            ] """

    def forward(self, x):
        return self.linear(x)

In [547]:
class DenseGrowNet(nn.Module):        
    def __init__(self, input_size, output_size, is_first_model):
        super(DenseGrowNet, self).__init__()
        self.activation = nn.ReLU()
        
        self.input_size = input_size
        self.output_size = output_size
        
        self.is_first_model = is_first_model
                        
        if is_first_model:
            self.first_layer = CustomLinearLayer(input_size, output_size, init="zero")
        else:
            self.first_layer = CustomLinearLayer(input_size, input_size * 2, init="looks_linear")
            self.last_layer = CustomLinearLayer(input_size * 2, output_size, init="zero")

    def forward(self, x, prev_output=None):
        if self.is_first_model and prev_output is not None:
            raise ValueError("This is the first model and prev_output is passed in")
        if not self.is_first_model and prev_output is None:
            raise ValueError("This is not the first model and prev_output is not passed in")
        
        if prev_output is None:
            return self.first_layer(x)
        
        x = self.activation(self.first_layer(x))
        return self.last_layer(x) + prev_output

In [500]:
num_epochs = 100
num_features = 100
num_classes = 2

model = DenseGrowNet(num_features, num_classes, True).to(device)
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, batch_size=1024, validation_size=0.2, raw_output=None)

criterion = CustomLoss(criterion=nn.CrossEntropyLoss(), l1_lambda=0.0001 * 0, l2_lambda=0.0001 * 0)
optimizer = optim.Adam(model.parameters(), lr=0.002)

for name, param in model.named_parameters():
    break
    print(f"Layer: {name}")
    print(f"Shape: {param.shape}")
    print(param)
    
summary(model, input_size=(1, num_features))
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, 5)

Epoch 1, Loss: 0.5845796328324538
Validation Loss: 0.5187402784824371
Val Accuracy: 0.7475, Val F1-score: 0.7655

Epoch 2, Loss: 0.49597448874742556
Validation Loss: 0.4743054836988449
Val Accuracy: 0.7804, Val F1-score: 0.7946

Epoch 3, Loss: 0.45988934544416576
Validation Loss: 0.444704607129097
Val Accuracy: 0.8045, Val F1-score: 0.8150

Epoch 4, Loss: 0.4332095170632387
Validation Loss: 0.42139087021350863
Val Accuracy: 0.8220, Val F1-score: 0.8292

Epoch 5, Loss: 0.41191143026718724
Validation Loss: 0.40256697237491607
Val Accuracy: 0.8331, Val F1-score: 0.8379

Epoch 6, Loss: 0.3946922413813762
Validation Loss: 0.38731867372989653
Val Accuracy: 0.8395, Val F1-score: 0.8424

Epoch 7, Loss: 0.38075525714800906
Validation Loss: 0.3749644041061401
Val Accuracy: 0.8424, Val F1-score: 0.8437

Epoch 8, Loss: 0.36946373337354416
Validation Loss: 0.36494105458259585
Val Accuracy: 0.8467, Val F1-score: 0.8470

Epoch 9, Loss: 0.36029337919675386
Validation Loss: 0.3567859947681427
Val Accur

In [548]:
X_tensor = torch.tensor(x_scaled).float()

x_scaled_1 = None
raw_output = None

with torch.no_grad():
    raw_output = model(X_tensor)
    raw_output_np = raw_output.detach().numpy()
        
    x_scaled_1 = np.concatenate((x_scaled, raw_output_np), axis=1)
    x_scaled_1 = x_scaler.fit_transform(x_scaled_1)
    
print(raw_output.shape)
print(x_scaled_1.shape)

torch.Size([48842, 2])
(48842, 102)


In [549]:
num_features = 102

model1 = DenseGrowNet(num_features, num_classes, False).to(device)
custom_train_loader = CustomDataLoader(x_scaled_1, y_encoded, batch_size=1024, validation_size=0.2, raw_output=raw_output)

optimizer = optim.Adam(model1.parameters(), lr=0.0001)
evaluate_model(model1, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, patience=5)

torch.Size([9769, 2])
Epoch 1, Loss: 0.3136664529641469
Validation Loss: 0.3159541130065918
Val Accuracy: 0.8561, Val F1-score: 0.8507

torch.Size([9769, 2])
Epoch 2, Loss: 0.3135322561630836
Validation Loss: 0.3158873528242111
Val Accuracy: 0.8562, Val F1-score: 0.8507

torch.Size([9769, 2])
Epoch 3, Loss: 0.3134492161946419
Validation Loss: 0.31581225991249084
Val Accuracy: 0.8566, Val F1-score: 0.8512

torch.Size([9769, 2])
Epoch 4, Loss: 0.31335888917629534
Validation Loss: 0.3157337337732315
Val Accuracy: 0.8563, Val F1-score: 0.8509

torch.Size([9769, 2])
Epoch 5, Loss: 0.3132594365340013
Validation Loss: 0.3156507819890976
Val Accuracy: 0.8562, Val F1-score: 0.8508

torch.Size([9769, 2])
Epoch 6, Loss: 0.31314825247495603
Validation Loss: 0.3155656546354294
Val Accuracy: 0.8560, Val F1-score: 0.8506

torch.Size([9769, 2])
Epoch 7, Loss: 0.3130245560254806
Validation Loss: 0.3154775172472
Val Accuracy: 0.8560, Val F1-score: 0.8507

torch.Size([9769, 2])
Epoch 8, Loss: 0.312888609

In [551]:
X_tensor_1 = torch.tensor(x_scaled_1).float()
print(X_tensor_1.shape)

x_scaled_2 = None
raw_output_1 = None

with torch.no_grad():
    raw_output_1 = model1(X_tensor_1, raw_output)
    print(raw_output_1.shape)
    raw_output_np = raw_output.detach().numpy()
        
    x_scaled_2 = np.concatenate((x_scaled_1, raw_output_np), axis=1)
    x_scaled_2 = x_scaler.fit_transform(x_scaled_2)
    
print(x_scaled_2.shape)
print(raw_output_1.shape)

torch.Size([48842, 102])
torch.Size([48842, 2])
(48842, 104)
torch.Size([48842, 2])


In [568]:
num_features = 104

model2 = DenseGrowNet(num_features, num_classes, False).to(device)
custom_train_loader = CustomDataLoader(x_scaled_2, y_encoded, batch_size=1024, validation_size=0.2, raw_output=raw_output_1)

optimizer = optim.Adam(model2.parameters(), lr=0.00006)
evaluate_model(model2, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, patience=5)

torch.Size([9769, 2])
Epoch 1, Loss: 0.29336777787942153
Validation Loss: 0.3087498605251312
Val Accuracy: 0.8590, Val F1-score: 0.8543

torch.Size([9769, 2])
Epoch 2, Loss: 0.2933294008939694
Validation Loss: 0.30874393582344056
Val Accuracy: 0.8591, Val F1-score: 0.8543

torch.Size([9769, 2])
Epoch 3, Loss: 0.2933221535804944
Validation Loss: 0.30873301029205324
Val Accuracy: 0.8590, Val F1-score: 0.8542

torch.Size([9769, 2])
Epoch 4, Loss: 0.2933142093511728
Validation Loss: 0.3087207943201065
Val Accuracy: 0.8590, Val F1-score: 0.8542

torch.Size([9769, 2])
Epoch 5, Loss: 0.2933058242003123
Validation Loss: 0.3087082266807556
Val Accuracy: 0.8590, Val F1-score: 0.8542

torch.Size([9769, 2])
Epoch 6, Loss: 0.2932970157036415
Validation Loss: 0.3086954325437546
Val Accuracy: 0.8589, Val F1-score: 0.8541

torch.Size([9769, 2])
Epoch 7, Loss: 0.2932876623593844
Validation Loss: 0.3086824119091034
Val Accuracy: 0.8590, Val F1-score: 0.8542

torch.Size([9769, 2])
Epoch 8, Loss: 0.293277